In [1]:
import os

print(f'Current working directory: {os.getcwd()}')
parent_dir = os.path.dirname(os.getcwd())
print(f'Parent directory: {parent_dir}')
os.chdir(parent_dir)
print(f'Current working directory: {os.getcwd()}')
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload

# notebook will reload external python modules;
%autoreload 2 

Current working directory: d:\azeem\Documents\UNIGE\MSc CS\Semester IV\METL\ner_seq2seq_project\notebooks
Parent directory: d:\azeem\Documents\UNIGE\MSc CS\Semester IV\METL\ner_seq2seq_project
Current working directory: d:\azeem\Documents\UNIGE\MSc CS\Semester IV\METL\ner_seq2seq_project


In [16]:
# from src.neural_baseline import *
from neural_baseline import *
from src.utils.conlleval import *

# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload

# notebook will reload external python modules;
%autoreload 2 

# make figures appear inline
# %matplotlib inline 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### util functions

In [17]:
def calculate_metrics(dataset, ner_model, mapping):
    all_true_tag_ids, all_predicted_tag_ids = [], []
    
    for x, y in dataset:
        output = ner_model.predict(x, verbose=0)  # set verbose to 0
        predictions = np.argmax(output, axis=-1)
        predictions = np.reshape(predictions, [-1])

        true_tag_ids = np.reshape(y, [-1])

        mask = (true_tag_ids > 0) & (predictions > 0)
        true_tag_ids = true_tag_ids[mask]
        predicted_tag_ids = predictions[mask]

        all_true_tag_ids.append(true_tag_ids)
        all_predicted_tag_ids.append(predicted_tag_ids)

    all_true_tag_ids = np.concatenate(all_true_tag_ids)
    all_predicted_tag_ids = np.concatenate(all_predicted_tag_ids)

    predicted_tags = [mapping[tag] for tag in all_predicted_tag_ids]
    real_tags = [mapping[tag] for tag in all_true_tag_ids]
    
    res = evaluate(real_tags, predicted_tags, verbose = True)

    return res

## Baseline n°1: Neural Network Implementation with Keras

In [18]:
vocab_size = 20000
batch_size = 32
epochs = 10
sample_text = "eu rejects german call to boycott british lamb"


print(f"processing data and preparing vocabulary of size {vocab_size}...")    
conll_data = load_and_prepare_data()

mapping = make_tag_lookup_table()

# vocab_size = 20000
vocabulary = get_vocabulary(conll_data, vocab_size)

print(f"preparing datasets...")
lookup_layer = keras.layers.StringLookup(vocabulary=vocabulary)

# batch_size = 32
train_dataset, val_dataset = prepare_datasets(vocabulary, batch_size)

num_tags = len(mapping)

print(f"creating model...\n")
ner_model = create_model(num_tags, vocab_size)

print(f"training model...\n")
compile_and_fit(ner_model, train_dataset, epochs=epochs)

processing data and preparing vocabulary of size 20000...


Found cached dataset conll2003 (C:/Users/azeem/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

preparing datasets...
creating model...

training model...

Epoch 1/10
439/439 [==============================] - 11s 19ms/step - loss: 0.6282
Epoch 2/10
439/439 [==============================] - 7s 17ms/step - loss: 0.2512
Epoch 3/10
439/439 [==============================] - 9s 20ms/step - loss: 0.1540
Epoch 4/10
439/439 [==============================] - 7s 17ms/step - loss: 0.1191
Epoch 5/10
439/439 [==============================] - 7s 16ms/step - loss: 0.0957
Epoch 6/10
439/439 [==============================] - 7s 16ms/step - loss: 0.0786
Epoch 7/10
439/439 [==============================] - 7s 15ms/step - loss: 0.0654
Epoch 8/10
439/439 [==============================] - 7s 16ms/step - loss: 0.0573
Epoch 9/10
439/439 [==============================] - 8s 17ms/step - loss: 0.0514
Epoch 10/10
439/439 [==============================] - 7s 16ms/step - loss: 0.0465


In [19]:
print(predict_sample(ner_model, sample_text, mapping, lookup_layer))

print(f"\ncalculating metrics...\n")
res = calculate_metrics(val_dataset, ner_model, mapping)

# res is a tuple of (precision, recall, f1), print it out beautifully
print("\n")
print(f"precision: \t{res[0]:.2f}")
print(f"   recall: \t{res[1]:.2f}")
print(f"       f1: \t{res[2]:.2f}")

1/1 [==============================] - 0s 220ms/step
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O']

calculating metrics...

processed 51362 tokens with 5942 phrases; found: 5219 phrases; correct: 3848.
accuracy:  62.55%; (non-O)
accuracy:  93.40%; precision:  73.73%; recall:  64.76%; FB1:  68.95
              LOC: precision:  83.97%; recall:  79.26%; FB1:  81.55  1734
             MISC: precision:  75.32%; recall:  64.53%; FB1:  69.51  790
              ORG: precision:  67.64%; recall:  58.61%; FB1:  62.80  1162
              PER: precision:  65.95%; recall:  54.89%; FB1:  59.91  1533


precision: 	73.73
   recall: 	64.76
       f1: 	68.95


## Baseline n°2: CRF Implementation with sklearn

In [2]:
%matplotlib inline
from src.crf_baseline import * 
from IPython.display import display

# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload

# notebook will reload external python modules;
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def read_raw_input(filename):
    """Read a train/test file and return the contents as a list of list of lists. 
    
    The innermost list is a record of 4 items, one per word.
    The middle-level list contains all the records in one sentence.
    """

    all_items = []

    with open(filename) as fh:
        current_item = []
        all_items.append(current_item)

        for line in fh:
            tags = line.strip().split()
            if len(tags) == 0 or tags[0] == '-DOCSTART-':
                continue
            current_item.append(tags)
            if tags[0] == '.' and tags[1] == '.':
                current_item = []
                all_items.append(current_item)
                
    return all_items

train_sents = read_raw_input('./data/CoNLL-2003_train.txt')
test_sents = read_raw_input('./data/CoNLL-2003_test.txt')

display(train_sents[0])

[['EU', 'NNP', 'B-NP', 'B-ORG'],
 ['rejects', 'VBZ', 'B-VP', 'O'],
 ['German', 'JJ', 'B-NP', 'B-MISC'],
 ['call', 'NN', 'I-NP', 'O'],
 ['to', 'TO', 'B-VP', 'O'],
 ['boycott', 'VB', 'I-VP', 'O'],
 ['British', 'JJ', 'B-NP', 'B-MISC'],
 ['lamb', 'NN', 'I-NP', 'O'],
 ['.', '.', 'O', 'O']]

In [4]:
train_sents = all_sentences(train_sents)
test_sents  = all_sentences(test_sents)
train_sents[0]

,word,pos,parse,ner
word_seq_num,,,,
0,EU,NNP,B-NP,B-ORG
1,rejects,VBZ,B-VP,O
2,German,JJ,B-NP,B-MISC
3,call,NN,I-NP,O
4,to,TO,B-VP,O
5,boycott,VB,I-VP,O
6,British,JJ,B-NP,B-MISC
7,lamb,NN,I-NP,O
8,.,.,O,O


In [5]:
X_train = get_feature_values(train_sents)
X_test = get_feature_values(test_sents)
y_train, y_test = get_labels(train_sents), get_labels(test_sents)

100%|██████████| 7375/7375 [03:00<00:00, 40.88it/s]


In [8]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    verbose=False,
    all_possible_transitions=True
)

crf.fit(X_train, y_train)

labels = list(crf.classes_)
labels.remove('O')
display(labels)

['B-ORG', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']

CPU times: total: 41.4 s
Wall time: 51.5 s


In [15]:
y_pred = crf.predict(X_test)
f1_score = metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)
print(f" flat f1 score: {f1_score:.2f}")

report = calculate_metrics_crf(y_test, y_pred, labels)
print(f"{report}")

 flat f1 score: 0.81


## Improvement Proposition